In [12]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
from datetime import datetime
import base as bs

import os as os
import csv

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [13]:
def get_titulo(url):
    try:
        titulo = url.find_elements(By.XPATH,'./a')[0].text
    except:
        return None
    else:
        return titulo

In [14]:
def get_autor(url):
    try:
        autor = url.find_element(By.XPATH, './/a[contains(@class, "c_a_a")]').text
    except:
        autor = None
    return autor

In [15]:
def get_ciudad(url):
    try:
        ciudad = url.find_element(By.XPATH, './/span[contains(@class, "c_a_l")]').text
    except:
        ciudad = None
    return ciudad

In [16]:
def get_fecha(url):
    try:
        fecha = url.find_element(By.XPATH, './/time').text.split()[:3]
        fecha_pub = " ".join(fecha)
    except:
        fecha_pub = None
    return fecha_pub

In [17]:
def get_resumen(url):
    try:
        resumen = url.find_element(By.XPATH, './/p').text
    except:
        resumen = None
    return resumen

In [69]:
def get_tema(url):
    try:
        tema = url.find_element(By.XPATH, './/span[contains(@class, "c_k")]').text
    except:
        tema = None
    return tema

In [81]:
def get_imag(url):
    try:
        imagen = url.find_element(By.XPATH, './/img').get_attribute('src')
    except:
        imagen = None
    return imagen

In [18]:
#Empresa con la cual vamos a extraer los articulos
empresa = input("Digite la empresa a extraer: ").lower()
revista = "elPais"

In [82]:
# crear driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://elpais.com/buscador/?q={empresa}')
time.sleep(2)

# Si en 5 segundo no ha encontrado el elemento que se le pasa se detiene
# Luego acepta los cookies
WebDriverWait(driver, 5)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      "button.didomi-components-button didomi-button didomi-dismiss-button didomi-components-button--color didomi-button-highlight highlight-button".replace(" ", "."))))\
                                          .click()
                                          

In [102]:
# Sacar primer titular CON BASE DE DATOS INICIAL
# titulares = []
# princip = driver.find_element(By.XPATH,'.//h2[contains(@class,"c_t ")]')
# urlPrinc = princip.find_elements(By.XPATH,'./a')[0].get_attribute('href')
# infPrinc = driver.find_element(By.XPATH,'.//div[contains(@class,"c_a")]')
# aux = driver.find_element(By.XPATH,'.//article[contains(@class,"c c-d c--m   ")]')
# if not(bs.existedb(urlPrinc, revista)):
#     tituloPrinc = get_titulo(princip)
#     autorPrinc = get_autor(infPrinc)
#     ciudadPrinc = get_ciudad(infPrinc)
#     fechaPrinc = get_fecha(infPrinc)
#     resumenPrinc = get_resumen(aux)
#     titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#                       'Titulo': tituloPrinc,
#                       'Fecha Publicacion':fechaPrinc,
#                       'URL': urlPrinc,
#                       'Resumen': resumenPrinc,
#                       'Autor': autorPrinc,
#                       'Ciudad Publicacion': ciudadPrinc})

In [83]:
# "c c-d _g _g-md c-m-l c--m-n" Contiene tema
# TODO: Tener en cuenta que mientras mas se baja mas articulos aparacen
articulos_Imag = driver.find_elements(By.XPATH, './/article[contains(@class, "c c-d _g _g-md c-m-l c--m")]')
articulos_NoImag = driver.find_elements(By.XPATH, './/article[contains(@class, "c c-d _g _g-md c-m-l c--m-n")]')
articulos = articulos_Imag + articulos_NoImag


In [88]:
titulares = []
for art in articulos:
    aux = art.find_element(By.XPATH, './/h2[contains(@class, "c_t")]')
    url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
    if not(bs.existedb(url, revista)):
        inf = art.find_element(By.XPATH,'.//div[contains(@class,"c_a")]')
        titulo = get_titulo(aux)
        autor = get_autor(inf)
        ciudad = get_ciudad(inf)
        fecha = get_fecha(inf)
        resumen = get_resumen(art)
        tema = get_tema(art)
        if art in articulos_Imag:
            imagen = get_imag(art)
        else:
            imagen = None
        titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'Titulo': titulo,
                        'Fecha Publicacion':fecha,
                        'Tema': tema,
                        'Resumen': resumen,
                        'URL': url,
                        'Imagen': imagen,
                        'Empresa': empresa,
                        'Autor': autor
                        # 'Contenido': contenido,
                        # 'Ciudad Publicacion': ciudad,
                        })

In [89]:
columnas = ['Fecha Extraccion', 'Titulo', 'Fecha Publicacion', 'Tema',
            'Resumen', 'URL', 'Imagen', 'Empresa', 'Autor']

In [66]:
# Para AGREGAR titulares a un archivo.
with open(f'../data/raw/{revista}.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [90]:
#Para crear un NUEVO archivo con los titulares
pd.json_normalize(titulares).to_csv(f'../data/raw/{revista}.csv',index=False, encoding='latin-1', errors='ignore', columns=columnas)